# Introduction


 I tried to predict the sentiment of tweets in Anaconda Kernel by using texts and basic Classifier .In this notebook I want to try whether we can outperform these model with a deep learning.
 
 
.fit a deep learning model with keras


.identify and deal with overfitting

.use word embeddings

.build on a pretrained model


# Set-up of the project

We'll start by importing some packages.

In [1]:
# Basic packages
import pandas as pd
import numpy as np
import re
import collections
import matplotlib.pyplot as plt
import seaborn as sns

# Packages for data preparation
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Packages for modeling
from keras import models
from keras import layers
from keras import regularizers

Using TensorFlow backend.


Then we set some parameters that will be used throughout the notebook.

In [2]:
NB_WORDS = 10000  # Parameter indicating the number of words we'll put in the dictionary
VAL_SIZE = 1000  # Size of the validation set
NB_START_EPOCHS = 20  # Number of epochs we usually start to train with
BATCH_SIZE = 512  # Size of the batches used in the mini-batch gradient descent

We read in the csv with  the tweets data and perform a random shuffle.It's a good prectice to shuffle data before spiliting between a train and test set.That way the sentiment classes ae equally distibuted over the train and test sets.

We will keep the text column as input and the airline_sentiment column as the target.

In [3]:
df=pd.read_csv('Tweets.csv')
display(df.shape)

display(df.head())

FileNotFoundError: [Errno 2] File Tweets.csv does not exist: 'Tweets.csv'

In [ ]:
df.columns

In [ ]:
df['tweet_id'].is_unique

In [ ]:
df=df.reindex(np.random.permutation(df.index))
df=df[['text','airline_sentiment']]
df.head()

# Data preparation 

Data Cleaning

The first thing we'll do is removing stopwords.These words do not have any value for predicting the sentiment.Furthermore,as we want build a model that can be used for other airline comapnies as well,we remove the mentions.

In [ ]:
!pip  install nltk

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
def remove_mentions(input_text):
        return re.sub(r'@\w+', '', input_text)
       
df.text = df.text.apply(remove_stopwords).apply(remove_mentions)
df.head()

In [ ]:
df['airline_sentiment'].unique()

In [ ]:
sns.countplot(df['airline_sentiment'])

# Train-Test split

The evaluation of the model performance needs to be done on a separate test set.As such, we can estimate how well the model generalizes.This is done with the train_test_split method of scikit-learn.

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df.text,df.airline_sentiment,test_size=0.1,random_state=37)
print('# Train data samples:',X_train.shape[0])
print('#Test data samples:',X_test.shape[0])
assert X_train.shape[0]==y_train.shape[0]
assert X_test.shape[0]==y_test.shape[0]

# Converting words to numbers

To use the text as input for a model ,we first need to convert the tweet's words into tokens,which simply means converting the words to integers that refer to an index in a dictionary.Here we will only keep the most frequent words in the tain set.

We clean up the text by applying filters and putting the words to lowercase.Words are separated by spaces.

In [ ]:
tk = Tokenizer(num_words=NB_WORDS,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ")
tk.fit_on_texts(X_train)

print('Fitted tokenizer on {} documents'.format(tk.document_count))
print('{} words in dictionary'.format(tk.num_words))
print('Top 5 most common words are:', collections.Counter(tk.word_counts).most_common(5))

After having created the dictionary we can convert the text to list of integer indexes.This is done with the text_to_sequences method of the Tokenizer.

In [ ]:
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

print('"{}" is converted into {}'.format(X_train[0], X_train_seq[0]))
print('"{}" is converted into {}'.format(X_test[0], X_test_seq[0]))

These integers should now be converted into a one-hot encoded features.

In [ ]:
def one_hot_seq(seqs, nb_features = NB_WORDS):
    ohs = np.zeros((len(seqs), nb_features))
    for i, s in enumerate(seqs):
        ohs[i, s] = 1.
    return ohs

X_train_oh = one_hot_seq(X_train_seq)
X_test_oh = one_hot_seq(X_test_seq)

print('"{}" is converted into {}'.format(X_train_seq[0], X_train_oh[0]))
print('For this example we have {} features with a value of 1.'.format(X_train_oh[0].sum()))

# Converting the target classes to numbers

We need to convert the target classes to numbers as well,which in turn are one-hot-encoded with the to_categorical method in keras.

In [ ]:
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)

print('"{}" is converted into {}'.format(y_train[0], y_train_le[0]))
print('"{}" is converted into {}'.format(y_train_le[0], y_train_oh[0]))

# Splitting of a validation set

Now that our data is ready,we split of a validation set.This validation set will be used to evaluate the model performance when we tune the parameters of the model

In [ ]:
X_train_rest, X_valid, y_train_rest, y_valid = train_test_split(X_train_oh, y_train_oh, test_size=0.1, random_state=37)

assert X_valid.shape[0] == y_valid.shape[0]
assert X_train_rest.shape[0] == y_train_rest.shape[0]

print('Shape of validation set:',X_valid.shape)

# Deep learning

We start with a model with 2 densely connected layers of 64 hidden elements.The input_shape for the first layer is equal to the number of words we allowed in the dictionary and for which we created one-hot-encoded features.

As we need to predict 3 different sentiment classes,the last layer has 3 hiddden elements.The softmax activation function makes sure the three probabilites sum up to 1.

In [ ]:
base_model = models.Sequential()
base_model.add(layers.Dense(64, activation='relu', input_shape=(NB_WORDS,)))
base_model.add(layers.Dense(64, activation='relu'))
base_model.add(layers.Dense(3, activation='softmax'))
base_model.summary()

Because this project is a multi-class ,single-label prediction,we use categorical_crossentropy as the loss function and softmax as final activation function.We fit the model on the reamining train data and validate on the validation set.We run for predetermined number of epochs and will see when the model starts to overfit.

In [ ]:
def deep_model(model):
    model.compile(optimizer='rmsprop'
                  , loss='categorical_crossentropy'
                  , metrics=['accuracy'])
    
    history = model.fit(X_train_rest
                       , y_train_rest
                       , epochs=NB_START_EPOCHS
                       , batch_size=BATCH_SIZE
                       , validation_data=(X_valid, y_valid)
                       , verbose=0)
    
    return history

In [ ]:
base_history = deep_model(base_model)

To evaluate the model performance,we will look at the training and validation loss and accuracy.

In [ ]:
def eval_metric(history, metric_name):
    metric = history.history[metric_name]
    val_metric = history.history['val_' + metric_name]

    e = range(1, NB_START_EPOCHS + 1)

    plt.plot(e, metric, 'bo', label='Train ' + metric_name)
    plt.plot(e, val_metric, 'b', label='Validation ' + metric_name)
    plt.legend()
    plt.show()

We can see here that the validation loss starts to increase as from epoch 4.
The training loss continues to lower,which is normal as the model is trained to fit the train data as good as possible.

In [ ]:
eval_metric(base_history, 'loss')

Just as with validation loss, the validation accuracy peaks at an early epoch.After that,it goes down slightly.So to conclude,we can say that the model starts overfitting as from epoch 4.

In [ ]:
eval_metric(base_history,'accuracy')

# Handling overfitting

Now,we can try to do something about the overfitting.These are different options to do that:
    
    .Option 1:reduce the network's size by removing layers or reducing the number of hidden elements in the layers.
        
        .Option 2: add regularization,which comes down to adding a cost to the loss function for large weights
            
            .option 3: adding dropout layers, which will randomly remove certain features by setting them to zero
                
            
            
        

Reducing the network's size

We reduce the network's size by removing one layer and lowering the number of hidden element in the remaining layer to 32.

In [ ]:
reduced_model = models.Sequential()
reduced_model.add(layers.Dense(32, activation='relu', input_shape=(NB_WORDS,)))
reduced_model.add(layers.Dense(3, activation='softmax'))
reduced_model.summary()

In [ ]:
reduced_history = deep_model(reduced_model)

In [ ]:
def compare_loss_with_baseline(h, model_name):
    loss_base_model = base_history.history['val_loss']
    loss_model = h.history['val_loss']

    e = range(1, NB_START_EPOCHS + 1)

    plt.plot(e, loss_base_model, 'bo', label='Validation Loss Baseline Model')
    plt.plot(e, loss_model, 'b', label='Validation Loss ' + model_name)
    plt.legend()
    plt.show()

We can see that it takes more epochs before the reduced model starts overfitting(around epoch 10).Moreover,the loss increases much slower after that epoch compared to the baseline model.

In [ ]:
compare_loss_with_baseline(reduced_history, 'Reduced Model')

# Adding regularization



To address overfitting,we can also add regularization to the model.Let's try with L2 regularization.

In [ ]:
reg_model = models.Sequential()
reg_model.add(layers.Dense(64, kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(NB_WORDS,)))
reg_model.add(layers.Dense(64, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
reg_model.add(layers.Dense(3, activation='softmax'))
reg_model.summary()

In [ ]:
reg_history = deep_model(reg_model)

For the regularized model we notice that it starts overfitting earlier than the baseline model.However,the loss increases much slower afterwards.

In [ ]:
compare_loss_with_baseline(reg_history, 'Regularized Model')


# Adding dropout layers

The last option we'll try to add dropout layers.

In [ ]:
drop_model = models.Sequential()
drop_model.add(layers.Dense(64, activation='relu', input_shape=(NB_WORDS,)))
drop_model.add(layers.Dropout(0.5))
drop_model.add(layers.Dense(64, activation='relu'))
drop_model.add(layers.Dropout(0.5))
drop_model.add(layers.Dense(3, activation='softmax'))
drop_model.summary()

In [ ]:
drop_history = deep_model(drop_model)

The model with dropout layers starts overfitting a bit later than the baseline model.The loss also increases slower than the baseline model.

In [ ]:
compare_loss_with_baseline(drop_history, 'Dropout Model')

# Training on the full train data and evaluation on test data

At first sight the reduced model seems to be the best model for generalization.But let's check that on the test set.

In [ ]:
def test_model(model, epoch_stop):
    model.fit(X_train_oh
              , y_train_oh
              , epochs=epoch_stop
              , batch_size=BATCH_SIZE
              , verbose=0)
    results = model.evaluate(X_test_oh, y_test_oh)
    
    return results

In [ ]:
base_results = test_model(base_model, 4)
print('/n')
print('Test accuracy of baseline model: {0:.2f}%'.format(base_results[1]*100))

In [ ]:
reduced_results = test_model(reduced_model, 10)
print('/n')
print('Test accuracy of reduced model: {0:.2f}%'.format(reduced_results[1]*100))

In [ ]:
reg_results = test_model(reg_model, 5)
print('/n')
print('Test accuracy of regularized model: {0:.2f}%'.format(reg_results[1]*100))

In [ ]:
drop_results = test_model(drop_model, 6)
print('/n')
print('Test accuracy of dropout model: {0:.2f}%'.format(drop_results[1]*100))